### 전체 validation data의 Cosine Similarity

In [1]:
import json
from tqdm import tqdm

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer

import kss
import pickle

/opt/conda/envs/mrc/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path = "../data/hyunwoongko_kobart_v3_generated.pickle"

with open(data_path, "rb") as f:
    data = pickle.load(f)

In [7]:
data

,generated_output,ground_truth
0,부동산·편법거래 및 시장 교란행위가 부동산시장을 왜곡하고 있지만 기대심리와 투기수요...,국내 기관뿐만 아니라 BIS 등 국제기구에서도 과도하게 상승한 주택 가격의 조정 가...
1,정부는 코로나19 이후 증가한 시중 유동성에 대한 관리를 강화해나가고자 하며 경제의...,정부는 경제의 안정적 성장과 부동산시장 안정을 위해 가계부채를 보다 촘촘히 점검·관...
2,자녀 내에서 일·생활 균형문화 확산을 위해 가족친화인증제도의 인증 기준을 강화하는 ...,의료종사자의 인권침해 예방 등 여성이 많이 종사하는 직종에 대한 보호를 강화해나가고...
3,국립감염병연구소에서는 백신접종자의 혈액을 채취한 후 실제 바이러스에 노출시켜 감염을...,국립감염병연구소에서 백신 접종자의 혈액을 채취하여 중화항체 실험을 시행했다.
4,온벤처붐 안착과 성과 확산을 촉진하고 혁신 중소벤처기업 육성 및 지역 기반 창업 활...,중소벤처기업부는 전 부처 중소기업 정책의 총괄 기능을 강화하고 빅데이터 기반 정책 ...
...,...,...
7295,<불후의 명곡-전설을 노래하다> ‘들국화 전인권’ 편에서 박완규는 '불미스러운 첫 ...,<불후의 명곡-전설을 노래하다>에서 박완규는 전인권에게 고소를 당해 법정에서 첫 만...
7296,'살림남2'에 출연한 홍성흔은 코로나 확산으로 인해 재계약 불가로 국내에 머물고 있...,‘살림하는 남자들 시즌2’에 새롭게 출연한 홍성흔이 17년 만에 함께 살게 된 가족...
7297,'살림남2'에서 올림픽 양궁 2관왕 김제덕이 처음으로 등교를 했으며 담임 선생님과 ...,올해 첫 등교를 하게 된 올림픽 양궁 선수 김제덕의 일상이 예능 프로그램 ‘살림하는...
7298,'1박 2일' 시즌 4는 익숙한 재미와 톡톡 튀는 신선함으로 안방극장을 웃음바다에 ...,'1박2일' 시즌 4가 새 출연진과 제작진으로 재미를 선사했고 멤버들이 룰라의 노래...


In [4]:
def get_sentence_embedding(model, text: list, batch_size: int = 16) -> torch.Tensor:
    embeddings = []
    n_batch = len(text) // batch_size + 1
    for i in range(n_batch):
        with torch.no_grad():
            embedding = model.encode(
                sentences=text[batch_size * i : batch_size * (i + 1)],
                batch_size=batch_size,
                show_progress_bar=False,
                convert_to_tensor=True,
                device="cuda:0",
            )
        embeddings.append(embedding.cpu())
    return torch.cat(embeddings).squeeze()

In [5]:
def sentence_embedding_sim_matrix(generated_text: list, target_text: str) -> np.ndarray:
    '''
    reference: https://github.com/jhgan00/ko-sentence-transformers
    '''
    # model setting
    model = SentenceTransformer('jhgan/ko-sroberta-multitask')

    # get sbert embeddings
    generated_embedding = get_sentence_embedding(model, generated_text, batch_size=16)
    target_embedding = get_sentence_embedding(model, target_text, batch_size=16)

    # get similarity
    length = len(target_embedding)
    sim_matrix = torch.zeros(length, dtype=torch.float16)
    for i in tqdm(range(length), desc="Similarity Matrix"):
        similarity = F.cosine_similarity(target_embedding[i], generated_embedding)
        sim_matrix[i] += similarity[i].item()

    sim_matrix = sim_matrix.numpy()

    return sim_matrix

In [6]:
generated_text = data.loc[:, 'generated_output'].tolist()
target_text = data.loc[:, 'ground_truth'].tolist()
sim_scores = sentence_embedding_sim_matrix(generated_text, target_text)
sim_scores

Similarity Matrix: 100%|██████████| 7300/7300 [00:27<00:00, 261.80it/s]


array([0.5884, 0.8086, 0.6763, ..., 0.7065, 0.7266, 0.814 ], dtype=float16)

In [8]:
data['cosine_similarity'] = sim_scores
data

,generated_output,ground_truth,cosine_similarity
0,부동산·편법거래 및 시장 교란행위가 부동산시장을 왜곡하고 있지만 기대심리와 투기수요...,국내 기관뿐만 아니라 BIS 등 국제기구에서도 과도하게 상승한 주택 가격의 조정 가...,0.588379
1,정부는 코로나19 이후 증가한 시중 유동성에 대한 관리를 강화해나가고자 하며 경제의...,정부는 경제의 안정적 성장과 부동산시장 안정을 위해 가계부채를 보다 촘촘히 점검·관...,0.808594
2,자녀 내에서 일·생활 균형문화 확산을 위해 가족친화인증제도의 인증 기준을 강화하는 ...,의료종사자의 인권침해 예방 등 여성이 많이 종사하는 직종에 대한 보호를 강화해나가고...,0.676270
3,국립감염병연구소에서는 백신접종자의 혈액을 채취한 후 실제 바이러스에 노출시켜 감염을...,국립감염병연구소에서 백신 접종자의 혈액을 채취하여 중화항체 실험을 시행했다.,0.905273
4,온벤처붐 안착과 성과 확산을 촉진하고 혁신 중소벤처기업 육성 및 지역 기반 창업 활...,중소벤처기업부는 전 부처 중소기업 정책의 총괄 기능을 강화하고 빅데이터 기반 정책 ...,0.634277
...,...,...,...
7295,<불후의 명곡-전설을 노래하다> ‘들국화 전인권’ 편에서 박완규는 '불미스러운 첫 ...,<불후의 명곡-전설을 노래하다>에서 박완규는 전인권에게 고소를 당해 법정에서 첫 만...,0.837402
7296,'살림남2'에 출연한 홍성흔은 코로나 확산으로 인해 재계약 불가로 국내에 머물고 있...,‘살림하는 남자들 시즌2’에 새롭게 출연한 홍성흔이 17년 만에 함께 살게 된 가족...,0.696289
7297,'살림남2'에서 올림픽 양궁 2관왕 김제덕이 처음으로 등교를 했으며 담임 선생님과 ...,올해 첫 등교를 하게 된 올림픽 양궁 선수 김제덕의 일상이 예능 프로그램 ‘살림하는...,0.706543
7298,'1박 2일' 시즌 4는 익숙한 재미와 톡톡 튀는 신선함으로 안방극장을 웃음바다에 ...,'1박2일' 시즌 4가 새 출연진과 제작진으로 재미를 선사했고 멤버들이 룰라의 노래...,0.726562


In [9]:
data['cosine_similarity'].mean()

0.683

### Rouge 계산

- 여러 개 한번에 계산하려면 문장 단위 list로 넣으면 됨

In [10]:
from rouge import Rouge


model_out = ["봉준호 감독의 영화 <신자유주의로 인한 양극화>가 세계적인 보편성을 획득하여 칸영화제 황금종려상을 수상했다."]
reference = ["영화의 주제가 세계적인 공통 문제라 높은 평가를 받았고, 대중적이지 않은 줄거리지만 칸영화제 후광 효과가 작용했다."]
rouge = Rouge()
scores = rouge.get_scores(model_out, reference, avg=True)
print("ROUGE-1(F1)", round(scores['rouge-1']['f'], 3))
print("ROUGE-2(F1)", round(scores['rouge-2']['f'], 3))
print("ROUGE-L(F1)", round(scores['rouge-l']['f'], 3))

ROUGE-1(F1) 0.148
ROUGE-2(F1) 0.0
ROUGE-L(F1) 0.148


### 1개 데이터에 대한 Cosine Similarity

In [11]:
def get_sentence_embedding(model, text: list, batch_size: int = 16) -> torch.Tensor:
    with torch.no_grad():
        embedding = model.encode(
            sentences         = text,
            batch_size        = batch_size,
            show_progress_bar = False,
            convert_to_tensor = True,
            device            = 'cuda:0',
        )
    return embedding.cpu()

def sentence_embedding_sim_matrix(generated_text: list, target_text: str) -> np.ndarray:
    '''
    reference: https://github.com/jhgan00/ko-sentence-transformers
    '''
    # model setting
    model = SentenceTransformer('jhgan/ko-sroberta-multitask')

    # get sbert embeddings
    generated_embedding = get_sentence_embedding(model, generated_text, batch_size=1)
    target_embedding = get_sentence_embedding(model, target_text, batch_size=1)

    # get similarity
    similarity = F.cosine_similarity(target_embedding, generated_embedding)

    return similarity

In [12]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('jhgan/ko-sroberta-multitask')

generated_embedding = get_sentence_embedding(model, model_out, batch_size=1)
target_embedding = get_sentence_embedding(model, reference, batch_size=1)
similarity = F.cosine_similarity(target_embedding, generated_embedding)
similarity

tensor([0.6405])